In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("MAT")
Pkg.add("Printf")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("JLD")
Pkg.add("Plots")

using JuMP
using Gurobi
using MAT
using Printf
using DataFrames
using CSV
using Plots
using JLD

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed FFMPEG_jll ─ v4.4.2+2
  No Changes to `~/.julia/environments/v1.8/Project.toml`
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [b22a6f82] ↑ FFMPEG_jll v4.4.2+1 ⇒ v4.4.2+2
Precompiling project...
  ✓ FFMPEG_jll
  ✓ FFMPEG
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  5 dependencies successfully precompiled in 23 seconds. 173 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Change

In [146]:
# High-level Settings
Zone = "NORTH" # price zone name

# read RTP price
fileln = matopen(string("./RTP_",Zone,"_2019_2021_1h.mat"))
RTP = read(fileln, string("RTP_",Zone,"_2019_2021_1h"))
close(fileln)
# read DAP price
fileln = matopen(string("./DAP_",Zone,"_2019_2021.mat"))
DAP = read(fileln, string("DAP_",Zone,"_2019_2021_Julia"))
close(fileln)

# read Discharge bid price1
fileln = matopen(string("./lambda_disb1_",Zone,".mat"))
disb_price1 = read(fileln, string("lambda_disb1"))
close(fileln)
# read Discharge bid price2
fileln = matopen(string("./lambda_disb2_",Zone,".mat"))
disb_price2 = read(fileln, string("lambda_disb2"))
close(fileln)

# read charge bid price1
fileln = matopen(string("./lambda_chrb1_",Zone,".mat"))
chrb_price1 = read(fileln, string("lambda_chrb1"))
close(fileln)
# read charge bid price2
fileln = matopen(string("./lambda_chrb2_",Zone,".mat"))
chrb_price2 = read(fileln, string("lambda_chrb2"))
close(fileln)

disb_price1 = reshape(disb_price1,(24,365))
disb_price2 = reshape(disb_price2,(24,365))
chrb_price1 = reshape(chrb_price1,(24,365))
chrb_price2 = reshape(chrb_price2,(24,365))

# RTP = reshape(RTP,(8760,1));
# # RTP = RTP[1:24]
# DAP = reshape(DAP,(8760,1));
# # DAP = DAP[1:24];

24×365 Matrix{Float64}:
  16.0787     23.1669    28.2047   …   16.3141     0.618492   15.822
  15.4296     25.2388    25.7656       11.6972    -3.53201     7.79401
  19.032      19.3603    22.2354        7.83809   -2.46551     6.05251
   4.55248    18.8767    16.1647        8.23484   -2.58251     8.38276
  10.2895     20.4606     4.54718       8.30159   -4.00601     9.77485
   2.61246    15.9838    13.4009   …   12.7197    -2.22346     8.21345
   9.08859    22.2741    14.5288       14.7153    -0.785496    7.56469
  16.903      14.6414    15.7706       17.0739    -3.91951     6.65974
   1.91804    15.6699    18.101        12.3947    -4.18411     7.85154
   0.150127   14.6392    18.2118       13.2889     1.88749    11.0563
  20.0273     15.4051    23.4115   …   10.2533   -51.0293     13.5632
  22.2485     19.7234    19.1432       13.5899   -32.4045     15.1998
   8.26537    26.6551    22.2574       12.3987   -32.7563     16.4086
  12.1261     30.3256    26.385        12.5901   -34.3493  

In [147]:
## battery setting

E = 1;  # storage energy capacity in MWh
# simulation setting
T = 24; # total time steps
P = .5; # power rating MW
eta = 0.9; # efficiency
e0 = .5 * E;
ef = e0;
MC = 10; # marginal discharge cost
M = 1; # duaration per step in hour
N_sim = 365; # number of days
d_DAP = DAP[:,1]; # daily DA price
d_RTP = RTP[:,1];
d_disb_price1 = disb_price1[:,1];
d_chrb_price1 = chrb_price1[:,1];
d_disb_price2 = disb_price2[:,1];
d_chrb_price2 = chrb_price2[:,1];

# initialize optimization model
model = Model(Gurobi.Optimizer)
set_silent(model) # no outputs

@variable(model, d_da[1:T], lower_bound = 0) # discharge power in DA
# @variable(model, d_rt[1:T], lower_bound = 0) # discharge power in RT
@variable(model, c_da[1:T], lower_bound = 0) # charge power in DA
# @variable(model, c_rt[1:T], lower_bound = 0) # charge power in RT
# @variable(model, diff[1:T]) # difference between RT and DA power decision

@variable(model, e[1:T], lower_bound = 0) # total battery energy
@variable(model, R) # market revenue
# @variable(model, C) # total degradation cost
@variable(model, u_da[1:T], Bin) # 1 if charge/buy in DAM

# @constraint(model, DiffPower[t=1:T], diff[t] - d_rt[t] + c_rt[t] == -d_pDA[t] ) # actual dis/charge power 

# arbitrage revenue
@constraint(model, ArbRev, M*sum( (d_DAP .- d_disb_price1).*d_da + (d_chrb_price1 .- d_DAP).*c_da ) - R == 0 )
# @constraint(model, ArbRev, M*sum( (d_DAP .- d_disb_price2).*d_da + (d_chrb_price2 .- d_DAP).*c_da ) - R == 0 )

# total degradation cost
# @constraint(model, DegCost, M*sum(d_rt * MC) - C == 0 )
# @constraint(model, DegCost, C == M*sum(diff .* MC))

# initial SoC evolution
@constraint(model, SoCInit, e[1] - e0 == M*(c_da[1]*eta - d_da[1]/eta) )
# rest SoC evolution
@constraint(model, SoCCont[t = 2:T], e[t] - e[t-1] == M*(c_da[t]*eta - d_da[t]/eta) )

# final energy level
@constraint(model, Enelast, e[T] >= ef )
# @constraint(model, Enelast[t = 24:24:T], e[t] >= ef )

# charging / discharging non-conflict condition
@constraint(model, ChRatingTot[t=1:T], c_da[t] <= P*u_da[t])
@constraint(model, DchRatingTot[t=1:T], d_da[t] <= P*(1-u_da[t]))

# max energy level
@constraint(model, SoCMax[t=1:T], e[t] <= E )

# maximize revenue plus degradation value
@objective(model, Max, R);
# @objective(model, Max, R);
# print(model);
# ArbRev


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [148]:
# initialize
R_s = zeros(1, N_sim)
P_s = zeros(1, N_sim)
C_s = zeros(1, N_sim)

C_v_da = zeros(24, N_sim) # daily DA charge
D_v_da = zeros(24, N_sim) # daily DA discharge
soc = zeros(24, N_sim) # daily SoC

# step = 0

@time begin
@printf("Optimization starts...\n")

for n = 1:(N_sim)       
    # update prices
    local pda = DAP[:,n]
    local prt = RTP[:,n]
    local pdisb1 = disb_price1[:,n]
    local pchrb1 = chrb_price1[:,n]
    local pdisb2 = disb_price2[:,n]
    local pchrb2 = chrb_price2[:,n]
        
    # update prices in constraints
    for t = 1:T
        set_normalized_coefficient(ArbRev, d_da[t], M*(pda[t]-pdisb1[t]))           
        set_normalized_coefficient(ArbRev, c_da[t], -M*(pda[t]-pchrb1[t]))     
        # set_normalized_coefficient(ArbRev, d_da[t], M*(pda[t]-pdisb2[t]))           
        # set_normalized_coefficient(ArbRev, c_da[t], -M*(pda[t]-pchrb2[t]))           
    end

    optimize!(model)
    
    global R_s[n] = value(R)
    # global C_s[n] = value(C)
    global P_s[n] = objective_value(model)
    # global P_s[n] = objective_value(model)

    global C_v_da[:,n] = value.(c_da)*M
    global D_v_da[:,n] = value.(d_da)*M    
    global soc[:,n] = value.(e)
    termination_status(model)
    # global e0 = value.(e)[end]
        
    # @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, Cummulative Cost %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), sum(C_s), termination_status(model))
    # @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), termination_status(model))
 
end
    
end
rtp = vec(reshape(RTP,(8760,1)))
dap = vec(reshape(DAP,(8760,1)))

DAcharge = vec(reshape(C_v_da,(8760,1)))
DAdischarge = vec(reshape(D_v_da,(8760,1)))
SoC = vec(reshape(soc,(8760,1)))
df = DataFrame(DAP = dap, RTP = rtp, DA_Charge = DAcharge, DA_Discharge = DAdischarge, SoC = SoC)
# CSV.write("DAdispatch_918.csv", df);
CSV.write(string("DA_Economic_Dispatch1_",Zone,"_2019.csv"), df);
# CSV.write(string("DA_Economic_Dispatch2_",Zone,"_2019.csv"), df);



Optimization starts...
  1.147979 seconds (617.63 k allocations: 28.826 MiB)


In [149]:
C_v_da

24×365 Matrix{Float64}:
 0.0        0.0  0.0       0.5       …  0.5  0.5        0.5       0.5
 0.0555556  0.5  0.5       0.0          0.0  0.0555556  0.0       0.0
 0.5        0.0  0.0       0.0          0.5  0.0        0.179012  0.0
 0.0        0.5  0.5       0.5          0.0  0.0        0.0       0.0
 0.234568   0.5  0.0       0.0          0.0  0.0        0.0       0.111111
 0.0        0.0  0.0       0.111111  …  0.5  0.0        0.5       0.5
 0.5        0.5  0.0       0.5          0.0  0.5        0.5       0.5
 0.5        0.0  0.0       0.5          0.5  0.5        0.0       0.0
 0.0        0.0  0.5       0.0          0.5  0.0        0.228395  0.0
 0.0        0.0  0.111111  0.5          0.0  0.5        0.5       0.5
 0.5        0.0  0.5       0.0       …  0.0  0.0        0.0       0.5
 0.5        0.0  0.0       0.240741     0.5  0.5        0.5       0.0
 0.0        0.5  0.0       0.0          0.5  0.117284   0.117284  0.5
 0.0        0.5  0.111111  0.5          0.0  0.0        0.0  

In [137]:
D_v_da

24×365 Matrix{Float64}:
 0.0   0.0    0.0   0.0    0.355  0.0   …  0.0    0.0    0.0    0.0    0.0
 0.0   0.0    0.26  0.36   0.0    0.26     0.0    0.5    0.0    0.5    0.5
 0.0   0.0    0.5   0.0    0.5    0.5      0.0    0.0    0.5    0.0    0.355
 0.5   0.0    0.0   0.4    0.0    0.0      0.405  0.26   0.4    0.5    0.0
 0.4   0.0    0.5   0.5    0.315  0.5      0.0    0.5    0.0    0.0    0.0
 0.0   0.405  0.0   0.0    0.0    0.0   …  0.0    0.0    0.0    0.0    0.0
 0.0   0.0    0.0   0.0    0.0    0.0      0.0    0.315  0.0    0.0    0.0
 0.0   0.0    0.0   0.0    0.5    0.0      0.0    0.0    0.0    0.0    0.5
 0.31  0.0    0.0   0.5    0.0    0.4      0.4    0.0    0.315  0.405  0.4
 0.5   0.0    0.0   0.0    0.0    0.5      0.5    0.4    0.0    0.0    0.0
 0.0   0.4    0.0   0.0    0.5    0.0   …  0.0    0.5    0.5    0.5    0.0
 0.0   0.5    0.5   0.0    0.0    0.0      0.0    0.0    0.0    0.0    0.215
 0.0   0.0    0.4   0.5    0.305  0.0      0.0    0.0    0.0    0.0    0

In [111]:
D_v_da - C_v_da  

24×365 Matrix{Float64}:
  0.0         0.0         0.0        …  -0.5        0.0       -0.5
 -0.0555556  -0.0555556   0.0            0.0        0.45       0.355
 -0.5         0.0         0.0            0.5        0.0        0.5
  0.5         0.0        -0.0617284      0.355      0.0        0.0
  0.0        -0.5         0.5            0.0        0.0        0.0
  0.31        0.405       0.0        …   0.0       -0.111111   0.0
 -0.5        -0.5         0.0           -0.111111  -0.5        0.0
 -0.5         0.0         0.0           -0.5        0.0        0.0
  0.4         0.0        -0.5            0.0        0.0        0.0
  0.5         0.0        -0.111111      -0.5       -0.5       -0.5
 -0.5         0.31       -0.5        …   0.0        0.5       -0.5
 -0.5         0.5         0.5            0.0       -0.5        0.0
  0.0         0.0         0.4            0.0       -0.117284  -0.111111
  0.0        -0.5         0.0            0.0        0.4        0.4
  0.5         0.0         0.0  

In [112]:
DAPP = reshape(DAP,(8760,1))
P_v_da = reshape(D_v_da - C_v_da,(8760,1))
sum(DAPP .* P_v_da)

-7744.77144691358

In [97]:
a = [1 2 3]
b= [ 3 8 10]'
r = a * b
# DAP = DAP[1:24];
c = [12 33]

1×2 Matrix{Int64}:
 12  33

In [98]:
string('c')

"c"